In [1]:
import torchaudio
import torch 
import pandas as pd
import numpy as np
import re 
import os 
from torch.nn import functional as F
import random
import math
import pickle
import gc
import torch.nn as nn
from tqdm.notebook import tqdm
import os
import glob
import warnings
from torch.utils.data import Dataset, DataLoader
warnings.filterwarnings(action='ignore')

In [2]:
base_dir = '/home/work/'
np1 = glob.glob(base_dir+'mel_dataset1/*.npy')
np2 = glob.glob(base_dir+'mel_dataset2/*.npy')
np3 = glob.glob(base_dir+'mel_dataset3/*.npy')
np4 = glob.glob(base_dir+'mel_dataset4/*.npy')
np5 = glob.glob(base_dir+'mel_dataset5/*.npy')
np6 = glob.glob(base_dir+'mel_dataset6/*.npy')
np7 = glob.glob(base_dir+'mel_dataset7/*.npy')
np8 = glob.glob(base_dir+'mel_dataset8/*.npy')

In [3]:
total = np1+np2+np3+np4+np5+np6+np7+np8

In [4]:
train_csv = pd.read_csv('new_train.csv')
valid_csv = pd.read_csv('new_valid.csv')
all_csv =pd.read_csv('new_all.csv')

In [5]:
def npy_list(csv):
    npy_list = []
    for song_path in tqdm(csv['npy_path']):
        song_npy = np.load(song_path).squeeze()
        #npy_list.append(song_npy[-1876:])
        npy_list.append(song_npy)
        
    return npy_list

train_list = [np.load(song_path).squeeze() for song_path in tqdm(train_csv['npy_path'])]
valid_list = [np.load(song_path).squeeze() for song_path in tqdm(valid_csv['npy_path'])]
#train_list = npy_list(train_csv)
#valid_list = npy_list(valid_csv)


  0%|          | 0/3825 [00:00<?, ?it/s]

  0%|          | 0/675 [00:00<?, ?it/s]

In [6]:
all_list = train_list + valid_list

In [7]:

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print(DEVICE)


cuda


In [ ]:

# epoch문 안에 들어가야되나?
#train_batch_li = DataLoader(train_list, batch_size=batch_size, shuffle=True,drop_last=True)
#val_batch_li = DataLoader(valid_list, batch_size=batch_size, shuffle=True,drop_last=True)


In [8]:
def train(model, train_loader):
    model.train()
    train_loss = 0
    for batch in train_loader: 
        
        
        mel = torch.FloatTensor(batch).to(DEVICE)
    
        optimizer.zero_grad()
        
        encode, output = model(mel)
        
        loss = criterion(output, mel)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
    train_loss /= (len(train_loader))

    return train_loss

def val(model, train_loader):
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in train_loader:
            
            
            mel = torch.FloatTensor(batch).to(DEVICE)
            
            encode, output = model(mel)

            loss = criterion(output, mel)
            
            val_loss += loss.item()

    val_loss /= (len(train_loader))

    return val_loss


def get_mel_embeding(model, train_loader):
    model.eval()
    mel_embeding_li = []
    with torch.no_grad():
        for batch in train_loader: 
            
            mel = torch.FloatTensor(batch).to(DEVICE)
            
            encode, output = model(mel)
            mel_embeding_li.append(encode.detach().cpu().numpy())

    return mel_embeding_li

In [10]:
#residual 을 어디다가 이어야될까...

In [11]:
'''
# 이거아님
############ 원래 ##############
class TimeAutoEncoder(nn.Module):
    def __init__(self):
        super(TimeAutoEncoder, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels = 48, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )
    
        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            #nn.BatchNorm1d(256),
            nn.ReLU(),
        )
            
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.conv5 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.conv6 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 8, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.conv7 = nn.Sequential(
            nn.Conv1d(in_channels = 16, out_channels = 4, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            #nn.BatchNorm1d(8),
            nn.ReLU(),
        )

        self.encoder_fc = nn.Sequential(
            nn.Linear(8 * 7501, 256),
            #nn.BatchNorm1d(256),
            nn.Tanh(),
            
        )
        
        self.decoder_fc = nn.Sequential(
            nn.Linear(256, 8 * 7501),
            nn.ReLU(),
        )

        self.t_conv1 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 8, out_channels = 16, kernel_size  = 3, stride = 1, dilation=62),
            nn.Conv1d(in_channels = 16, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            #nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.t_conv2 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 16, out_channels = 32, kernel_size  = 3, stride = 1, dilation = 30),
            nn.Conv1d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.t_conv3 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 32, out_channels = 64, kernel_size  = 3, stride = 1, dilation=14),
            nn.Conv1d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.t_conv4 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 64, out_channels = 128, kernel_size  = 3, stride = 1, dilation = 6),
            nn.Conv1d(in_channels = 128, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.t_conv5 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 128, out_channels = 256, kernel_size  = 3, stride = 1, dilation=2),
            nn.Conv1d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(256),
            nn.ReLU(),
        )

        self.t_conv6 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 256, out_channels = 512, kernel_size  = 3, stride = 1, dilation = 1),
            nn.Conv1d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.t_conv7 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 512, out_channels = 48, kernel_size  = 3, stride = 1, dilation= 1),
            nn.Conv1d(in_channels = 1024, out_channels = 48, kernel_size = 3, stride = 1, padding = 0, dilation = 1)
        )

        
        
        
        self.conv1to3 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 128 * 7501
        
        self.conv2to4 = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 64 * 7501
        
        self.conv3to5 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 32 * 7501
        
        self.conv4to6 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 8, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 16 * 7501
        
        self.conv5to7 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 4, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 8 * 7501
        
        
        
    def forward(self, mel_spec):
        x = F.pad(mel_spec, pad = (2, 0, 0, 0))
        x1 = self.conv1(x) # 48 * 7501 => 512 * 7501 
        #print(x1.shape)
        
        x1to2 = F.pad(x1, pad = (4, 0, 0, 0)) #512 * 7501 
        x1to3 = F.pad(x1, pad = (8, 0, 0, 0)) #512 * 7501 
        #print('x1to2',x1to2.shape)
        #print('x1to3',x1to3.shape)
        
        
        x2 = self.conv2(x1to2) # 512 * 7501 => 256 * 7501
        #print(x2.shape)
        
        x2to3 = F.pad(x2, pad = (8, 0, 0, 0)) # 256 * 7501
        x2to4 = F.pad(x2, pad = (16, 0, 0, 0)) # 256 * 7501
        #print('x2to3 :',x2to3.shape)
        #print('x2to4 :',x2to3.shape)
        
        x3 = self.conv3(x2to3) # 256 * 7501 => 64 * 7501
        x3_connec = self.conv1to3(x1to3) # In : 512 * 7501 , out: 64 * 7501     128
        x3= torch.cat([x3,x3_connec],1) # 128 * 7501
        #print(x3.shape)
        
        
        x3to4 = F.pad(x3, pad = (16, 0, 0, 0)) # 128 * 7501
        x3to5 = F.pad(x3, pad = (32, 0, 0, 0)) # 128 * 7501
        #print('x3to4 :',x3to4.shape)
        #print('x3to5 :',x3to5.shape)
        
        x4 = self.conv4(x3to4) # 128 * 7501 => 32 * 7501
        x4_connec = self.conv2to4(x2to4) # In : 256 * 7501 , out : 32 * 7501 
        x4 = torch.cat([x4,x4_connec],1) # 64 * 7501
        #print(x4.shape)
        
        x4to5 = F.pad(x4, pad = (32, 0, 0, 0)) # 64 * 7501
        x4to6 = F.pad(x4, pad = (64, 0, 0, 0)) # 64 * 7501
        #print('x4to5 :',x4to5.shape)
        #print('x4to6 :',x4to6.shape)
        
        
        x5 = self.conv5(x4to5) # 64 * 7501 = > 16 * 7501 
        x5_connec = self.conv3to5(x3to5) # 128 * 7501 => 16 * 7501
        x5 = torch.cat([x5,x5_connec],1) # 32 * 7501
        #print(x5.shape)
        
        x5to6 = F.pad(x5, pad = (64, 0, 0, 0)) # 32 * 7501
        x5to7 = F.pad(x5, pad = (128, 0, 0, 0)) # 32 * 7501
        #print('x5to6 :',x5to6.shape)
        #print('x5to7 :',x5to7.shape)
        
        
        x6 = self.conv6(x5to6) # 32 * 7501 => 8 * 7501
        x6_connec = self.conv4to6(x4to6) # 64 * 7501 => 8 * 7501
        x6 = torch.cat([x6,x6_connec],1) # 16 * 7501
        #print(x6.shape)
        x6to7 = F.pad(x6, pad = (128, 0, 0, 0)) # 16 * 7501
        #print('x6to7 :',x6to7.shape)
        
        
        x7 = self.conv7(x6to7) # 16 * 7501 => 4 *7501 
        x7_connec = self.conv5to7(x5to7) # 32 * 7501 => 4 * 7501
        x7 = torch.cat([x7,x7_connec],1) # 8 * 7501
        #print(x7.shape)
        encode = self.encoder_fc(x7.view(-1, 8 * 7501))

        #encode = self.encoder_fc(x.view(-1, 8 * 1876))

        # print('decode')
        x = self.decoder_fc(encode)
        x = x.view(-1, 8, 7501)
        #x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        
        x = torch.cat([x7,x],1) # 4,16,7501
        #x = self.concat7_shape(x) 
        x = F.pad(x, pad = (128, 0, 0, 0)) 
        x = self.t_conv1(x) # 4,8,7501
        #print(x.shape,'new_cat7')
        
        
        x = torch.cat([x6,x],1)
        #x = self.concat6_shape(x)
        x = F.pad(x, pad = (64, 0, 0, 0))
        x = self.t_conv2(x)
        
        
        x = torch.cat([x5,x],1)
        #x = self.concat5_shape(x)
        x = F.pad(x, pad = (32, 0, 0, 0))
        x = self.t_conv3(x)
        
        
        
        x = torch.cat([x4,x],1)
        #x = self.concat4_shape(x)
        x = F.pad(x, pad = (16, 0, 0, 0))
        x = self.t_conv4(x)
        
        
        x = torch.cat([x3,x],1)
        #x = self.concat3_shape(x)
        x = F.pad(x, pad = (8, 0, 0, 0))
        x = self.t_conv5(x)
        
        
        
        x = torch.cat([x2,x],1)
        #x = self.concat2_shape(x)
        x = F.pad(x, pad = (4, 0, 0, 0))
        x = self.t_conv6(x)
        
        
        
        x = torch.cat([x1,x],1)
        #x = self.concat1_shape(x)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.t_conv7(x)
        
        #print(x.shape)
        #x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        
        return encode, x
'''

"\n# 이거아님\n############ 원래 ##############\nclass TimeAutoEncoder(nn.Module):\n    def __init__(self):\n        super(TimeAutoEncoder, self).__init__()\n        self.conv1 = nn.Sequential(\n            nn.Conv1d(in_channels = 48, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 1),\n            #nn.BatchNorm1d(512),\n            nn.ReLU(),\n        )\n    \n        self.conv2 = nn.Sequential(\n            nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 2),\n            #nn.BatchNorm1d(256),\n            nn.ReLU(),\n        )\n            \n        self.conv3 = nn.Sequential(\n            nn.Conv1d(in_channels = 256, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 4),\n            #nn.BatchNorm1d(128),\n            nn.ReLU(),\n        )\n\n        self.conv4 = nn.Sequential(\n            nn.Conv1d(in_channels = 128, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 

In [9]:
'''
# beta
# 뭔가 잘못됐다
# swap 지우면 안되나?
#저장모델
############# 원래 ##############
class TimeAutoEncoder(nn.Module):
    def __init__(self):
        super(TimeAutoEncoder, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels = 48, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )
    
        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            #nn.BatchNorm1d(256),
            nn.ReLU(),
        )
            
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.conv5 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.conv6 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.conv7 = nn.Sequential(
            nn.Conv1d(in_channels = 16, out_channels = 4, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            #nn.BatchNorm1d(8),
            nn.ReLU(),
        )

        self.encoder_fc = nn.Sequential(
            nn.Linear(8 * 7501, 256),
            #nn.BatchNorm1d(128),
            nn.LeakyReLU(),
        )
        
        self.decoder_fc = nn.Sequential(
            nn.Linear(256, 8 * 7501),
            nn.ReLU(),
        )

        self.t_conv1 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 8, out_channels = 16, kernel_size  = 3, stride = 1, dilation=62),
            nn.Conv1d(in_channels = 16, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            #nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.t_conv2 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 16, out_channels = 32, kernel_size  = 3, stride = 1, dilation = 30),
            nn.Conv1d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.t_conv3 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 32, out_channels = 64, kernel_size  = 3, stride = 1, dilation=14),
            nn.Conv1d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.t_conv4 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 64, out_channels = 128, kernel_size  = 3, stride = 1, dilation = 6),
            nn.Conv1d(in_channels = 128, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.t_conv5 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 128, out_channels = 256, kernel_size  = 3, stride = 1, dilation=2),
            nn.Conv1d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(256),
            nn.ReLU(),
        )

        self.t_conv6 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 256, out_channels = 512, kernel_size  = 3, stride = 1, dilation = 1),
            nn.Conv1d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.t_conv7 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 512, out_channels = 48, kernel_size  = 3, stride = 1, dilation= 1),
            nn.Conv1d(in_channels = 1024, out_channels = 48, kernel_size = 3, stride = 1, padding = 0, dilation = 1)
        )

        
        
        
        self.conv1to3 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 128 * 7501
        
        self.conv2to4 = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 64 * 7501
        
        self.conv3to5 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 32 * 7501
        
        self.conv4to6 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 8, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 16 * 7501
        
        self.conv5to7 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 4, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 8 * 7501
        
        
        
    def forward(self, mel_spec):
        x = F.pad(mel_spec, pad = (2, 0, 0, 0))
        x1 = self.conv1(x) # 48 * 7501 => 512 * 7501 
        #print(x1.shape)
        
        x1to2 = F.pad(x1, pad = (4, 0, 0, 0)) #512 * 7501 
        x1to3 = F.pad(x1, pad = (8, 0, 0, 0)) #512 * 7501 
        #print('x1to2',x1to2.shape)
        #print('x1to3',x1to3.shape)
        
        
        x2 = self.conv2(x1to2) # 512 * 7501 => 256 * 7501
        #print(x2.shape)
        
        x2to3 = F.pad(x2, pad = (8, 0, 0, 0)) # 256 * 7501
        #x2to4 = F.pad(x2, pad = (16, 0, 0, 0)) # 256 * 7501
        #print('x2to3 :',x2to3.shape)
        #print('x2to4 :',x2to3.shape)
        
        x3 = self.conv3(x2to3) # 256 * 7501 => 64 * 7501
        x3_connec = self.conv1to3(x1to3) # In : 512 * 7501 , out: 64 * 7501     128
        x3= torch.cat([x3,x3_connec],1) # 128 * 7501
        #print(x3.shape)
        
        
        x3to4 = F.pad(x3, pad = (16, 0, 0, 0)) # 128 * 7501
        x3to5 = F.pad(x3, pad = (32, 0, 0, 0)) # 128 * 7501
        #print('x3to4 :',x3to4.shape)
        #print('x3to5 :',x3to5.shape)
        
        x4 = self.conv4(x3to4) # 128 * 7501 => 64 * 7501
        #x4_connec = self.conv2to4(x2to4) # In : 256 * 7501 , out : 32 * 7501 
        #x4 = torch.cat([x4,x4_connec],1) # 64 * 7501
        #print(x4.shape)
        
        x4to5 = F.pad(x4, pad = (32, 0, 0, 0)) # 64 * 7501
        #x4to6 = F.pad(x4, pad = (64, 0, 0, 0)) # 64 * 7501
        #print('x4to5 :',x4to5.shape)
        #print('x4to6 :',x4to6.shape)
        
        
        x5 = self.conv5(x4to5) # 64 * 7501 = > 16 * 7501 
        x5_connec = self.conv3to5(x3to5) # 128 * 7501 => 16 * 7501
        x5 = torch.cat([x5,x5_connec],1) # 32 * 7501
        #print(x5.shape)
        
        x5to6 = F.pad(x5, pad = (64, 0, 0, 0)) # 32 * 7501
        x5to7 = F.pad(x5, pad = (128, 0, 0, 0)) # 32 * 7501
        #print('x5to6 :',x5to6.shape)
        #print('x5to7 :',x5to7.shape)
        
        
        x6 = self.conv6(x5to6) # 32 * 7501 => 16 * 7501
        #x6_connec = self.conv4to6(x4to6) # 64 * 7501 => 8 * 7501
        #x6 = torch.cat([x6,x6_connec],1) # 16 * 7501
        #print(x6.shape)
        x6to7 = F.pad(x6, pad = (128, 0, 0, 0)) # 32 * 7501
        #print('x6to7 :',x6to7.shape)
        
        
        x7 = self.conv7(x6to7) # 16 * 7501 => 4 *7501 
        pre_encode = torch.flatten(x7)
        x7_connec = self.conv5to7(x5to7) # 32 * 7501 => 4 * 7501
        x7 = torch.cat([x7,x7_connec],1) # 8 * 7501
        #print(x7.shape)
        encode = self.encoder_fc(x7.view(-1, 8 * 7501))

        #encode = self.encoder_fc(x.view(-1, 8 * 1876))

        # print('decode')
        x = self.decoder_fc(encode)
        x = x.view(-1, 8, 7501)
        
        #x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        
        x = torch.cat([x7,x],1) # 4,16,7501
        #x = self.concat7_shape(x) 
        x = F.pad(x, pad = (128, 0, 0, 0)) 
        x = self.t_conv1(x) # 4,8,7501
        #print(x.shape,'new_cat7')
        
        
        x = torch.cat([x6,x],1)
        #x = self.concat6_shape(x)
        x = F.pad(x, pad = (64, 0, 0, 0))
        x = self.t_conv2(x)
        
        
        x = torch.cat([x5,x],1)
        #x = self.concat5_shape(x)
        x = F.pad(x, pad = (32, 0, 0, 0))
        x = self.t_conv3(x)
        
        
        
        x = torch.cat([x4,x],1)
        #x = self.concat4_shape(x)
        x = F.pad(x, pad = (16, 0, 0, 0))
        x = self.t_conv4(x)
        
        
        x = torch.cat([x3,x],1)
        #x = self.concat3_shape(x)
        x = F.pad(x, pad = (8, 0, 0, 0))
        x = self.t_conv5(x)
        
        
        
        x = torch.cat([x2,x],1)
        #x = self.concat2_shape(x)
        x = F.pad(x, pad = (4, 0, 0, 0))
        x = self.t_conv6(x)
        
        
        
        x = torch.cat([x1,x],1)
        #x = self.concat1_shape(x)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.t_conv7(x)
        
        #print(x.shape)
        #x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        
        return encode, x
'''

In [ ]:
'''
# bravo 저장
############# 원래 ##############
class TimeAutoEncoder(nn.Module):
    def __init__(self):
        super(TimeAutoEncoder, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels = 48, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )
    
        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            #nn.BatchNorm1d(256),
            nn.ReLU(),
        )
            
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.conv5 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.conv6 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.conv7 = nn.Sequential(
            nn.Conv1d(in_channels = 16, out_channels = 4, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            #nn.BatchNorm1d(8),
            nn.ReLU(),
        )

        self.encoder_fc = nn.Sequential(
            nn.Linear(8 * 7501, 256),
            #nn.BatchNorm1d(128),
            nn.LeakyReLU(),
        )
        
        self.decoder_fc = nn.Sequential(
            nn.Linear(256, 8 * 7501),
            nn.ReLU(),
        )

        self.t_conv1 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 8, out_channels = 16, kernel_size  = 3, stride = 1, dilation=62),
            nn.Conv1d(in_channels = 16, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            #nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.t_conv2 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 16, out_channels = 32, kernel_size  = 3, stride = 1, dilation = 30),
            nn.Conv1d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.t_conv3 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 32, out_channels = 64, kernel_size  = 3, stride = 1, dilation=14),
            nn.Conv1d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.t_conv4 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 64, out_channels = 128, kernel_size  = 3, stride = 1, dilation = 6),
            nn.Conv1d(in_channels = 128, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.t_conv5 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 128, out_channels = 256, kernel_size  = 3, stride = 1, dilation=2),
            nn.Conv1d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(256),
            nn.ReLU(),
        )

        self.t_conv6 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 256, out_channels = 512, kernel_size  = 3, stride = 1, dilation = 1),
            nn.Conv1d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.t_conv7 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 512, out_channels = 48, kernel_size  = 3, stride = 1, dilation= 1),
            nn.Conv1d(in_channels = 1024, out_channels = 48, kernel_size = 3, stride = 1, padding = 0, dilation = 1)
        )

        
        
        
        self.conv1to3 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 128 * 7501
        
        self.conv2to4 = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 64 * 7501
        
        self.conv3to5 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 32 * 7501
        
        self.conv4to6 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 8, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 16 * 7501
        
        self.conv5to7 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 4, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 8 * 7501
        
        
        
    def forward(self, mel_spec):
        x = F.pad(mel_spec, pad = (2, 0, 0, 0))
        x1 = self.conv1(x) # 48 * 7501 => 512 * 7501 
        #print(x1.shape)
        
        x1to2 = F.pad(x1, pad = (4, 0, 0, 0)) #512 * 7501 
        x1to3 = F.pad(x1, pad = (8, 0, 0, 0)) #512 * 7501 
        #print('x1to2',x1to2.shape)
        #print('x1to3',x1to3.shape)
        
        
        x2 = self.conv2(x1to2) # 512 * 7501 => 256 * 7501
        #print(x2.shape)
        
        x2to3 = F.pad(x2, pad = (8, 0, 0, 0)) # 256 * 7501
        #x2to4 = F.pad(x2, pad = (16, 0, 0, 0)) # 256 * 7501
        #print('x2to3 :',x2to3.shape)
        #print('x2to4 :',x2to3.shape)
        
        x3 = self.conv3(x2to3) # 256 * 7501 => 64 * 7501
        x3_connec = self.conv1to3(x1to3) # In : 512 * 7501 , out: 64 * 7501     128
        x3= torch.cat([x3,x3_connec],1) # 128 * 7501
        #print(x3.shape)
        
        
        x3to4 = F.pad(x3, pad = (16, 0, 0, 0)) # 128 * 7501
        x3to5 = F.pad(x3, pad = (32, 0, 0, 0)) # 128 * 7501
        #print('x3to4 :',x3to4.shape)
        #print('x3to5 :',x3to5.shape)
        
        x4 = self.conv4(x3to4) # 128 * 7501 => 64 * 7501
        #x4_connec = self.conv2to4(x2to4) # In : 256 * 7501 , out : 32 * 7501 
        #x4 = torch.cat([x4,x4_connec],1) # 64 * 7501
        #print(x4.shape)
        
        x4to5 = F.pad(x4, pad = (32, 0, 0, 0)) # 64 * 7501
        #x4to6 = F.pad(x4, pad = (64, 0, 0, 0)) # 64 * 7501
        #print('x4to5 :',x4to5.shape)
        #print('x4to6 :',x4to6.shape)
        
        
        x5 = self.conv5(x4to5) # 64 * 7501 = > 16 * 7501 
        x5_connec = self.conv3to5(x3to5) # 128 * 7501 => 16 * 7501
        x5 = torch.cat([x5,x5_connec],1) # 32 * 7501
        #print(x5.shape)
        
        x5to6 = F.pad(x5, pad = (64, 0, 0, 0)) # 32 * 7501
        x5to7 = F.pad(x5, pad = (128, 0, 0, 0)) # 32 * 7501
        #print('x5to6 :',x5to6.shape)
        #print('x5to7 :',x5to7.shape)
        
        
        x6 = self.conv6(x5to6) # 32 * 7501 => 16 * 7501
        #x6_connec = self.conv4to6(x4to6) # 64 * 7501 => 8 * 7501
        #x6 = torch.cat([x6,x6_connec],1) # 16 * 7501
        #print(x6.shape)
        x6to7 = F.pad(x6, pad = (128, 0, 0, 0)) # 32 * 7501
        #print('x6to7 :',x6to7.shape)
        
        
        x7 = self.conv7(x6to7) # 16 * 7501 => 4 *7501 
        pre_encode = torch.flatten(x7)
        x7_connec = self.conv5to7(x5to7) # 32 * 7501 => 4 * 7501
        x7 = torch.cat([x7,x7_connec],1) # 8 * 7501
        #print(x7.shape)
        encode = self.encoder_fc(x7.view(-1, 8 * 7501))

        #encode = self.encoder_fc(x.view(-1, 8 * 1876))

        # print('decode')
        x = self.decoder_fc(encode)
        x = x.view(-1, 8, 7501)
        
        #x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        
        x = torch.cat([x7,x],1) # 4,16,7501
        #x = self.concat7_shape(x) 
        x = F.pad(x, pad = (128, 0, 0, 0)) 
        x = self.t_conv1(x) # 4,8,7501
        #print(x.shape,'new_cat7')
        
        
        x = torch.cat([x6,x],1)
        #x = self.concat6_shape(x)
        x = F.pad(x, pad = (64, 0, 0, 0))
        x = self.t_conv2(x)
        
        
        x = torch.cat([x5,x],1)
        #x = self.concat5_shape(x)
        x = F.pad(x, pad = (32, 0, 0, 0))
        x = self.t_conv3(x)
        
        
        
        x = torch.cat([x4,x],1)
        #x = self.concat4_shape(x)
        x = F.pad(x, pad = (16, 0, 0, 0))
        x = self.t_conv4(x)
        
        
        x = torch.cat([x3,x],1)
        #x = self.concat3_shape(x)
        x = F.pad(x, pad = (8, 0, 0, 0))
        x = self.t_conv5(x)
        
        
        
        x = torch.cat([x2,x],1)
        #x = self.concat2_shape(x)
        x = F.pad(x, pad = (4, 0, 0, 0))
        x = self.t_conv6(x)
        
        
        
        x = torch.cat([x1,x],1)
        #x = self.concat1_shape(x)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.t_conv7(x)
        
        #print(x.shape)
        #x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        
        return encode, x
'''

In [85]:
'''
#bravo 노저장
############# 원래 ##############
class TimeAutoEncoder(nn.Module):
    def __init__(self):
        super(TimeAutoEncoder, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels = 48, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )
    
        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            #nn.BatchNorm1d(256),
            nn.ReLU(),
        )
            
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        
        self.encoder_fc = nn.Sequential(
            nn.Linear(16 * 7501, 1024),
            nn.LeakyReLU(),
            #nn.Linear(1024,256),
            #nn.BatchNorm1d(128),
            #nn.ReLU(),
        )
        
        self.decoder_fc = nn.Sequential(
            #nn.Linear(256, 1024),
            #nn.ReLU(),
            nn.Linear(1024, 16 * 7501),
            nn.ReLU(),
        
        )

        self.t_conv1 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 8, out_channels = 16, kernel_size  = 3, stride = 1, dilation=62),
            nn.Conv1d(in_channels = 16, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.t_conv2 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 16, out_channels = 32, kernel_size  = 3, stride = 1, dilation = 30),
            nn.Conv1d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.t_conv3 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 32, out_channels = 64, kernel_size  = 3, stride = 1, dilation=14),
            nn.Conv1d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.t_conv4 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 64, out_channels = 128, kernel_size  = 3, stride = 1, dilation = 6),
            nn.Conv1d(in_channels = 128, out_channels = 48, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        
        
        
        
        
        
    def forward(self, mel_spec):
        x = F.pad(mel_spec, pad = (2, 0, 0, 0))
        
        x1 = self.conv1(x) # 48 * 7501 => 128 * 7501 
        
        x1to2 = F.pad(x1, pad = (4, 0, 0, 0)) #512 * 7501 
        
        x2 = self.conv2(x1to2) # 128 * 7501 => 64 * 7501
        
        x2to3 = F.pad(x2, pad = (8, 0, 0, 0)) # 64 * 7501
        
        x3 = self.conv3(x2to3) # 64 * 7501 => 32 * 7501
        
        x3to4 = F.pad(x3, pad = (16, 0, 0, 0)) # 32 * 7501
        
        x4 = self.conv4(x3to4) # 32 * 7501 => 16 * 7501
        
        encode = self.encoder_fc(x4.view(-1, 16 * 7501))

        
        # print('decode')
        x = self.decoder_fc(encode)
        t_x0 = x.view(-1, 16, 7501)
        
        x = self.t_conv1(t_x0) # in: 16 * 7501 , out : 16 * 7485
        x = F.pad(x, pad = (16, 0, 0, 0)) # 16 * 7501
        t_x1 = torch.cat([x4,x],1) # 32 * 7501
        
      
        x = self.t_conv2(t_x1) # in : 32 * 7501 , out : 32 *  
        x = F.pad(x, pad = (8, 0, 0, 0)) 
        t_x2 = torch.cat([x3,x],1)
        
        
        x = self.t_conv3(t_x2) # 64 * 7501
        x = F.pad(x, pad = (4, 0, 0, 0))
        t_x3 = torch.cat([x2,x],1) # 128 * 7501
        
        
        x = F.pad(t_x3, pad = (2, 0, 0, 0))
        x = self.t_conv4(x) 
        
        return encode, x
'''

In [9]:
#charlie
############# 원래 ##############
class TimeAutoEncoder(nn.Module):
    def __init__(self):
        super(TimeAutoEncoder, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels = 48, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )
    
        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            #nn.BatchNorm1d(256),
            nn.ReLU(),
        )
            
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.conv5 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.conv6 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.conv7 = nn.Sequential(
            nn.Conv1d(in_channels = 16, out_channels = 4, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            #nn.BatchNorm1d(8),
            nn.ReLU(),
        )

        self.encoder_fc = nn.Sequential(
            nn.Linear(8 * 7501, 256),
            #nn.BatchNorm1d(128),
            nn.LeakyReLU(),
        )
        
        self.decoder_fc = nn.Sequential(
            nn.Linear(256, 8 * 7501),
            nn.ReLU(),
        )

        self.t_conv1 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 8, out_channels = 16, kernel_size  = 3, stride = 1, dilation=62),
            nn.Conv1d(in_channels = 16, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            #nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.t_conv2 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 16, out_channels = 32, kernel_size  = 3, stride = 1, dilation = 30),
            nn.Conv1d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            #nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.t_conv3 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 32, out_channels = 64, kernel_size  = 3, stride = 1, dilation=14),
            nn.Conv1d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.t_conv4 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 64, out_channels = 128, kernel_size  = 3, stride = 1, dilation = 6),
            nn.Conv1d(in_channels = 128, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.t_conv5 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 128, out_channels = 256, kernel_size  = 3, stride = 1, dilation=2),
            nn.Conv1d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(256),
            nn.ReLU(),
        )

        self.t_conv6 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 256, out_channels = 512, kernel_size  = 3, stride = 1, dilation = 1),
            nn.Conv1d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.t_conv7 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 512, out_channels = 48, kernel_size  = 3, stride = 1, dilation= 1),
            nn.Conv1d(in_channels = 1024, out_channels = 48, kernel_size = 3, stride = 1, padding = 0, dilation = 64)
        )

        
        
        
        self.conv1to3 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 128 * 7501
        
        self.conv2to4 = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 64 * 7501
        
        self.conv3to5 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 32 * 7501
        
        self.conv4to6 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 8, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 16 * 7501
        
        self.conv5to7 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 4, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 8 * 7501
        
        
        
    def forward(self, mel_spec):
        x = F.pad(mel_spec, pad = (2, 0, 0, 0))
        x1 = self.conv1(x) # 48 * 7501 => 512 * 7501 
        #print(x1.shape)
        
        x1to2 = F.pad(x1, pad = (4, 0, 0, 0)) #512 * 7501 
        x1to3 = F.pad(x1, pad = (8, 0, 0, 0)) #512 * 7501 
        #print('x1to2',x1to2.shape)
        #print('x1to3',x1to3.shape)
        
        
        x2 = self.conv2(x1to2) # 512 * 7501 => 256 * 7501
        #print(x2.shape)
        
        x2to3 = F.pad(x2, pad = (8, 0, 0, 0)) # 256 * 7501
        #x2to4 = F.pad(x2, pad = (16, 0, 0, 0)) # 256 * 7501
        #print('x2to3 :',x2to3.shape)
        #print('x2to4 :',x2to3.shape)
        
        x3 = self.conv3(x2to3) # 256 * 7501 => 64 * 7501
        x3_connec = self.conv1to3(x1to3) # In : 512 * 7501 , out: 64 * 7501     128
        x3= torch.cat([x3,x3_connec],1) # 128 * 7501
        #print(x3.shape)
        
        
        x3to4 = F.pad(x3, pad = (16, 0, 0, 0)) # 128 * 7501
        x3to5 = F.pad(x3, pad = (32, 0, 0, 0)) # 128 * 7501
        #print('x3to4 :',x3to4.shape)
        #print('x3to5 :',x3to5.shape)
        
        x4 = self.conv4(x3to4) # 128 * 7501 => 64 * 7501
        #x4_connec = self.conv2to4(x2to4) # In : 256 * 7501 , out : 32 * 7501 
        #x4 = torch.cat([x4,x4_connec],1) # 64 * 7501
        #print(x4.shape)
        
        x4to5 = F.pad(x4, pad = (32, 0, 0, 0)) # 64 * 7501
        #x4to6 = F.pad(x4, pad = (64, 0, 0, 0)) # 64 * 7501
        #print('x4to5 :',x4to5.shape)
        #print('x4to6 :',x4to6.shape)
        
        
        x5 = self.conv5(x4to5) # 64 * 7501 = > 16 * 7501 
        x5_connec = self.conv3to5(x3to5) # 128 * 7501 => 16 * 7501
        x5 = torch.cat([x5,x5_connec],1) # 32 * 7501
        #print(x5.shape)
        
        x5to6 = F.pad(x5, pad = (64, 0, 0, 0)) # 32 * 7501
        x5to7 = F.pad(x5, pad = (128, 0, 0, 0)) # 32 * 7501
        #print('x5to6 :',x5to6.shape)
        #print('x5to7 :',x5to7.shape)
        
        
        x6 = self.conv6(x5to6) # 32 * 7501 => 16 * 7501
        #x6_connec = self.conv4to6(x4to6) # 64 * 7501 => 8 * 7501
        #x6 = torch.cat([x6,x6_connec],1) # 16 * 7501
        #print(x6.shape)
        x6to7 = F.pad(x6, pad = (128, 0, 0, 0)) # 32 * 7501
        #print('x6to7 :',x6to7.shape)
        
        
        x7 = self.conv7(x6to7) # 16 * 7501 => 4 *7501 
        pre_encode = torch.flatten(x7)
        x7_connec = self.conv5to7(x5to7) # 32 * 7501 => 4 * 7501
        x7 = torch.cat([x7,x7_connec],1) # 8 * 7501
        #print(x7.shape)
        encode = self.encoder_fc(x7.view(-1, 8 * 7501))

        #encode = self.encoder_fc(x.view(-1, 8 * 1876))

        # print('decode')
        x = self.decoder_fc(encode)
        x = x.view(-1, 8, 7501)
        
        #x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        
        x = torch.cat([x7,x],1) # 4,16,7501
        #x = self.concat7_shape(x) 
        x = F.pad(x, pad = (2, 0, 0, 0)) 
        x = self.t_conv1(x) # 4,8,7501
        #print(x.shape,'new_cat7')
        
        
        x = torch.cat([x6,x],1)
        #x = self.concat6_shape(x)
        x = F.pad(x, pad = (4, 0, 0, 0))
        x = self.t_conv2(x)
        
        
        x = torch.cat([x5,x],1)
        #x = self.concat5_shape(x)
        x = F.pad(x, pad = (8, 0, 0, 0))
        x = self.t_conv3(x)
        
        
        
        x = torch.cat([x4,x],1)
        #x = self.concat4_shape(x)
        x = F.pad(x, pad = (16, 0, 0, 0))
        x = self.t_conv4(x)
        
        
        x = torch.cat([x3,x],1)
        #x = self.concat3_shape(x)
        x = F.pad(x, pad = (32, 0, 0, 0))
        x = self.t_conv5(x)
        
        
        
        x = torch.cat([x2,x],1)
        #x = self.concat2_shape(x)
        x = F.pad(x, pad = (64, 0, 0, 0))
        x = self.t_conv6(x)
        
        
        
        x = torch.cat([x1,x],1)
        #x = self.concat1_shape(x)
        x = F.pad(x, pad = (128, 0, 0, 0))
        x = self.t_conv7(x)
        
        #print(x.shape)
        #x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        
        return encode, x


In [10]:
model = TimeAutoEncoder().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.MSELoss()

In [11]:
gc.collect()

26

In [ ]:

import time

model_dir = '/home/work/Tcae_apply/model_dir3/'
min_loss = 987654321

epochs = 150
batch_size = 32

train_batch_li = DataLoader(train_list, batch_size=batch_size, shuffle=True,drop_last=False)
val_batch_li = DataLoader(valid_list, batch_size=batch_size, shuffle=True,drop_last=False)

#tloss_list=[]
#vloss_list=[]

for epoch in tqdm(range(1, epochs + 1)):
    start = time.time()
    
    
    train_loss = train(model = model, train_loader = train_batch_li) 
    val_loss = val(model = model, train_loader = val_batch_li) 
    end = time.time()
    
    
    
    
    #tloss_list.append(train_loss)
    #vloss_list.append(val_loss)
    
    print(f'EPOCH:{epoch}, Train Loss:{train_loss}, Val Loss:{val_loss}, 학습 시간: {end - start}')
    if (val_loss < min_loss) & (val_loss < 10) & (train_loss < 15) :
        min_loss = val_loss
        torch.save(model.state_dict(), model_dir + f'TimeAutoEncoder_skipconnection_charlie_val.pt')
        print('모델 저장')
        


  0%|          | 0/150 [00:00<?, ?it/s]

EPOCH:1, Train Loss:110260.32459309897, Val Loss:11433.421253551136, 학습 시간: 78.66790819168091
EPOCH:2, Train Loss:7394.33687133789, Val Loss:3800.166681463068, 학습 시간: 71.63565707206726
EPOCH:3, Train Loss:2928.3070871988934, Val Loss:1872.6620871803977, 학습 시간: 57.03754138946533
EPOCH:4, Train Loss:1687.3794443766276, Val Loss:1260.5794039639559, 학습 시간: 45.16006374359131
EPOCH:5, Train Loss:1121.6509796142577, Val Loss:912.6757535067471, 학습 시간: 36.65596675872803
EPOCH:6, Train Loss:864.2644838968913, Val Loss:726.4970619895241, 학습 시간: 35.88951921463013
EPOCH:7, Train Loss:682.7494117736817, Val Loss:561.0816653858532, 학습 시간: 35.71298885345459
EPOCH:8, Train Loss:554.2170783996582, Val Loss:519.9623315984553, 학습 시간: 35.77355408668518
EPOCH:9, Train Loss:607.3109842936198, Val Loss:447.49970176003194, 학습 시간: 35.61007761955261
EPOCH:10, Train Loss:477.72823333740234, Val Loss:532.4633622602983, 학습 시간: 35.18342614173889
EPOCH:11, Train Loss:401.6184939066569, Val Loss:347.6201789162376, 학습 

In [15]:
#!pip install torchmetrics
#from torchmetrics import MeanAbsolutePercentageError

#model = TimeAutoEncoder().to(DEVICE)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
#criterion = MeanAbsolutePercentageError()


In [34]:
'''
import time

model_dir = '/home/work/Tcae_apply/model_dir3/'
min_loss = 987654321

epochs = 150
batch_size = 64

train_batch_li = DataLoader(train_list, batch_size=batch_size, shuffle=True,drop_last=True)
val_batch_li = DataLoader(valid_list, batch_size=batch_size, shuffle=True,drop_last=True)

#tloss_list=[]
#vloss_list=[]

for epoch in tqdm(range(1, epochs + 1)):
    start = time.time()
    
    
    train_loss = train(model = model, train_loader = train_batch_li) 
    val_loss = val(model = model, train_loader = val_batch_li) 
    end = time.time()
    
    
    #tloss_list.append(train_loss)
    #vloss_list.append(val_loss)
    
    print(f'EPOCH:{epoch}, Train Loss:{train_loss}, Val Loss:{val_loss}, 학습 시간: {end - start}')
    if val_loss < min_loss:
        min_loss = val_loss
        torch.save(model.state_dict(), model_dir + f'TimeAutoEncoder_skipconnection_beta_val.pt')
        print('모델 저장')
        
'''

"\nimport time\n\nmodel_dir = '/home/work/Tcae_apply/model_dir3/'\nmin_loss = 987654321\n\nepochs = 150\nbatch_size = 64\n\ntrain_batch_li = DataLoader(train_list, batch_size=batch_size, shuffle=True,drop_last=True)\nval_batch_li = DataLoader(valid_list, batch_size=batch_size, shuffle=True,drop_last=True)\n\n#tloss_list=[]\n#vloss_list=[]\n\nfor epoch in tqdm(range(1, epochs + 1)):\n    start = time.time()\n    \n    \n    train_loss = train(model = model, train_loader = train_batch_li) \n    val_loss = val(model = model, train_loader = val_batch_li) \n    end = time.time()\n    \n    \n    #tloss_list.append(train_loss)\n    #vloss_list.append(val_loss)\n    \n    print(f'EPOCH:{epoch}, Train Loss:{train_loss}, Val Loss:{val_loss}, 학습 시간: {end - start}')\n    if val_loss < min_loss:\n        min_loss = val_loss\n        torch.save(model.state_dict(), model_dir + f'TimeAutoEncoder_skipconnection_beta_val.pt')\n        print('모델 저장')\n        \n"

In [12]:
model_dir = '/home/work/Tcae_apply/model_dir3/'

model = TimeAutoEncoder().to(DEVICE)
model.load_state_dict(torch.load(model_dir + f'TimeAutoEncoder_skipconnection_charlie_val.pt', map_location = DEVICE))

<All keys matched successfully>

In [13]:
test_batch_li = DataLoader(all_list, batch_size=1, shuffle=False,drop_last=False)


In [14]:
mel_embeding_li = get_mel_embeding(model = model, train_loader = test_batch_li)
mel_embeding = np.concatenate(mel_embeding_li, axis = 0)

In [15]:
pd.DataFrame(mel_embeding).to_csv('tcae_charlie_embedding.csv',index=False)

In [ ]:
#np.save(batch_data_dir + 'mel_embeding_val.npy', mel_embeding)

In [67]:
import glob

In [71]:
inference_npy_path = '/home/work/Tcae_apply/tcae_inference_folder/mel_folder/'
inference_npy_path = glob.glob(inference_npy_path + '*.npy')

In [72]:
inference_npy_path

['/home/work/Tcae_apply/tcae_inference_folder/mel_folder/성현_보컬.npy',
 '/home/work/Tcae_apply/tcae_inference_folder/mel_folder/정하_보컬.npy',
 '/home/work/Tcae_apply/tcae_inference_folder/mel_folder/지평_보컬.npy']

In [73]:
inference_list = [np.load(song_path).squeeze()[:,:7501] for song_path in tqdm(inference_npy_path)]

  0%|          | 0/3 [00:00<?, ?it/s]

In [77]:
inference_batch_li = DataLoader(inference_list, batch_size=1, shuffle=False,drop_last=False)

In [78]:
def get_mel_inference_embeding(model, train_loader):
    model.eval()
    mel_embeding_li = []
    with torch.no_grad():
        for batch in train_loader: 
            
            mel = torch.FloatTensor(batch).to(DEVICE)
 
            encode, output = model(mel)
            mel_embeding_li.append(encode.detach().cpu().numpy())

    return mel_embeding_li

In [79]:
inference_embedding_li = get_mel_embeding(model = model, train_loader = inference_batch_li)
inference_embedding = np.concatenate(inference_embedding_li, axis = 0)

In [80]:
pd.DataFrame(inference_embedding).to_csv('inference_embedding1.csv',index=False)